<a href="https://colab.research.google.com/github/benjaminMancilla/AS4501-Proyect/blob/main/Download_PS1_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install astro-delight
! pip install alerce

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 95.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import os, sys
import numpy as np
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
from alerce.core import Alerce
from delight.delight import *

In [8]:
!git config --global user.name "ConitoDC"
!git config --global user.email "constanza.dominguez@ug.uchile.cl"

In [9]:
!git clone https://github.com/benjaminMancilla/AS4501-Proyect.git

Cloning into 'AS4501-Proyect'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 30 (delta 9), reused 24 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 101.81 KiB | 2.83 MiB/s, done.
Resolving deltas: 100% (9/9), done.


In [3]:
# Abrimos los archivos xlsx usando pandas
veto1 = 'Vetoes.xlsx'
veto2 = 'Vetoes2.xlsx'

In [4]:
def extract_oids(archivo):
    oids = []
    sheets = pd.read_excel(archivo, sheet_name = None, skiprows=1)
    for df in sheets.values():
        if "oid" in df.columns:  # busca columna por nombre
            oids.extend(df["oid"].dropna().astype(str).tolist())
        else:  # si no existe, usa primera columna
            oids.extend(df.iloc[:, 0].dropna().astype(str).tolist())
    return oids

oids1 = extract_oids(veto1)
oids2 = extract_oids(veto2)
oids_total = oids1 + oids2

print('Hay', len(oids1), 'objetos en archivo de veto 1')
print('Hay', len(oids2), 'objetos en archivo de veto 2')
print('Hay', len(oids_total), 'objetos en total')

Hay 11239 objetos en archivo de veto 1
Hay 11988 objetos en archivo de veto 2
Hay 23227 objetos en total


In [ ]:
# Guardamos la lista de oids con np
np.save('oids_total.npy', oids_total)

In [ ]:
# Probamos una consulta con Alerce para ver cómo funciona
alerce_client = Alerce()

oid = "ZTF25aavgvqe"

object_info = alerce_client.query_objects(oid=oid, format="pandas")
print('Ra:', object_info['meanra'], 'Dec:', object_info['meandec'])
object_info

Ra: 0    212.28722
Name: meanra, dtype: float64 Dec: 0    31.147092
Name: meandec, dtype: float64


,oid,ndethist,ncovhist,mjdstarthist,mjdendhist,corrected,stellar,ndet,g_r_max,g_r_max_corr,...,lastmjd,deltajd,meanra,meandec,sigmara,sigmadec,class,classifier,probability,step_id_corr
0,ZTF25aavgvqe,1,2827,60842.245336,60842.245336,True,False,1,None,None,...,60842.245336,0.0,212.28722,31.147092,0.075949,0.065,None,None,None,27.5.6


In [ ]:
# Ocuparemos Alerce para encontrar los ra y dec de los objetos
alerce_client = Alerce()
data = []
errors = []

for oid in oids_total:
  try:
    obj_info = alerce_client.query_objects(oid=oid, format="pandas")
    ra = obj_info['meanra']
    dec = obj_info['meandec']
    data.append({'oid': oid, 'ra': ra, 'dec': dec})
  except Exception as e:
    print(f"Error con OID {oid}: {e}")
    errors.append(oid)
    data.append({'oid': oid, 'ra': None, 'dec': None})

print('--------------------')
print('Objetos encontrados:', len(data))
print('Objetos con error:', len(errors))
print('--------------------')
df = pd.DataFrame(data)
df

Error con OID ZTF22abnyduf	: 'meanra'
Error con OID ZTF22absmbfi	: 'meanra'
Error con OID ZTF23aailyfx : 'meanra'
Error con OID ZTF23aajvnae	: 'meanra'
Error con OID ZTF23aajduzt	: 'meanra'
Error con OID ZTF23aagrzdt	: 'meanra'
Error con OID ZTF23aagolfm	: 'meanra'
Error con OID  : 'meanra'
Error con OID  : 'meanra'
Error con OID  : 'meanra'
Error con OID  : 'meanra'
Error con OID  ZTF24aaqefva: 'meanra'
Error con OID ZTF25aaqosfn: {'Error code': 502, 'Message': 'Unknown API error.', 'Data': 'Unknown API error.'}
Error con OID ZTF25aaqyszt: {'Error code': 502, 'Message': 'Unknown API error.', 'Data': 'Unknown API error.'}
Error con OID ZTF25aaqkbuo: {'Error code': 502, 'Message': 'Unknown API error.', 'Data': 'Unknown API error.'}
Error con OID ZTF25aaqrcbp: {'Error code': 502, 'Message': 'Unknown API error.', 'Data': 'Unknown API error.'}
Error con OID ZTF25aaqowtc: {'Error code': 502, 'Message': 'Unknown API error.', 'Data': 'Unknown API error.'}
Error con OID ZTF25aaqkrze: {'Error c

,oid,ra,dec
0,ZTF22aaaolwx,"0 230.351159 Name: meanra, dtype: float64","0 54.250215 Name: meandec, dtype: float64"
1,ZTF22aaaxmix,"0 235.212955 Name: meanra, dtype: float64","0 13.226248 Name: meandec, dtype: float64"
2,ZTF22aabgasm,"0 244.949052 Name: meanra, dtype: float64","0 -8.071116 Name: meandec, dtype: float64"
3,ZTF22aabwuks,"0 203.220651 Name: meanra, dtype: float64","0 29.95307 Name: meandec, dtype: float64"
4,ZTF22aabwuae,"0 202.636665 Name: meanra, dtype: float64","0 30.546504 Name: meandec, dtype: float64"
...,...,...,...
23222,ZTF23abgnvya,"0 153.691004 Name: meanra, dtype: float64","0 36.686484 Name: meandec, dtype: float64"
23223,ZTF23abgoccj,"0 139.808046 Name: meanra, dtype: float64","0 14.69577 Name: meandec, dtype: float64"
23224,ZTF23abfubfc,"0 348.100839 Name: meanra, dtype: float64","0 17.71145 Name: meandec, dtype: float64"
23225,ZTF23abgaebf,"0 58.111949 Name: meanra, dtype: float64","0 74.808986 Name: meandec, dtype: float64"


In [ ]:
# Nos damos cuenta de un problema en las columnas de ra y dec
df["ra"] = df["ra"].astype(str).str.extract(r'(\d+\.\d+)').astype(float)
df["dec"] = df["dec"].astype(str).str.extract(r'(\d+\.\d+)').astype(float)

# Verificar resultado
print(df.head())

            oid          ra        dec
0  ZTF22aaaolwx  230.351159  54.250215
1  ZTF22aaaxmix  235.212955  13.226248
2  ZTF22aabgasm  244.949052   8.071116
3  ZTF22aabwuks  203.220651  29.953070
4  ZTF22aabwuae  202.636665  30.546504


In [ ]:
# Vemos qué ocurre con los objetos que dan error
alerce_client = Alerce()

oid = 'ZTF22abnyduf'

object_info = alerce_client.query_objects(oid=oid, format="pandas") # buscamos el obj con error con Alerce
print('Ra:', object_info['meanra'], 'Dec:', object_info['meandec'])
object_info

Ra: 0    15.016094
Name: meanra, dtype: float64 Dec: 0    21.505762
Name: meandec, dtype: float64


,oid,ndethist,ncovhist,mjdstarthist,mjdendhist,corrected,stellar,ndet,g_r_max,g_r_max_corr,...,lastmjd,deltajd,meanra,meandec,sigmara,sigmadec,class,classifier,probability,step_id_corr
0,ZTF22abnyduf,1,1216,59871.25,59871.25,False,False,1,None,None,...,59871.2989,0.0,15.016094,21.505762,None,None,None,None,None,1.1.6


In [ ]:
df[df['oid'] == 'ZTF22abnyduf'] # buscamos ahora el objeto en el df

,oid,ra,dec
85,ZTF22abnyduf,15.016094,21.505762


El error parecer ser causado por algun problema con Alerce, por lo cual se volverá a consultar por los oids donde se tuvo error y se agregarán al DataFrame para tener la lista completa de objetos.

In [ ]:
# Buscamos si hay valores NaN y guardamos sus oids
nan_oids = df[df.isna().any(axis=1)]['oid'].unique().tolist()
df_cop = df.copy()  # Copia del DataFrame original
successful_oids = []

for oid in nan_oids:
    try:
        obj_info = alerce_client.query_objects(oid=oid, format="pandas")
        ra = obj_info['meanra'].values[0]  # Valor escalar
        dec = obj_info['meandec'].values[0]

        # Actualiza las columnas 'ra' y 'dec' para el OID específico
        mask = df_cop['oid'] == oid
        df_cop.loc[mask, 'ra'] = ra
        df_cop.loc[mask, 'dec'] = dec

        print(f'Funcionó con OID {oid} | ra: {ra} | dec: {dec}')
        successful_oids.append(oid)
    except Exception as e:
        print(f"Error con OID {oid}: {e}")

# Verifica que los cambios se aplicaron correctamente
print("\nVerificación:")
print(df_cop[df_cop['oid'] == 'ZTF22abnyduf'][['oid', 'ra', 'dec']])


Funcionó con OID ZTF22abnyduf | ra: 15.0160936 | dec: 21.5057622
Funcionó con OID ZTF22absmbfi | ra: 183.28360235000002 | dec: 8.7117048
Funcionó con OID ZTF23aajvnae | ra: 202.60814750000003 | dec: 27.703520942857143
Funcionó con OID ZTF23aailyfx | ra: 288.2788276 | dec: -13.0880788
Funcionó con OID ZTF23aajduzt | ra: 170.79614485789472 | dec: 50.268018621052626
Funcionó con OID ZTF25aaqyszt | ra: 230.6586574492731 | dec: 15.978075072285048
Funcionó con OID ZTF25aaqosfn | ra: 222.36537968731764 | dec: 23.72494742137045
Funcionó con OID ZTF25aaqkbuo | ra: 311.9813202150717 | dec: 11.9676150220624
Funcionó con OID ZTF25aaqrcbp | ra: 165.11945839702807 | dec: 16.34331806063198
Funcionó con OID ZTF25aaqowtc | ra: 231.52138510000378 | dec: 16.048333000000003
Funcionó con OID ZTF25aaqkrze | ra: 291.4295776444406 | dec: 37.75325233902359
Funcionó con OID ZTF25aaggsfv | ra: 226.28232259812046 | dec: 14.504570867564668
Funcionó con OID ZTF20abhxdht | ra: 217.84889532107758 | dec: 64.3195003491

In [ ]:
# Nos damos cuenta que hay algunos repetidos, los eliminamos
df_sin_duplicados = df.copy()
df_sin_duplicados = df_cop.drop_duplicates()
df_sin_duplicados

,oid,ra,dec
0,ZTF22aaaolwx,230.351159,54.250215
1,ZTF22aaaxmix,235.212955,13.226248
2,ZTF22aabgasm,244.949052,8.071116
3,ZTF22aabwuks,203.220651,29.953070
4,ZTF22aabwuae,202.636665,30.546504
...,...,...,...
19965,ZTF23abhkkcv,20.503617,2.779283
19966,ZTF23abhfhze,13.003263,9.481944
19967,ZTF23abhdcct,259.063642,37.111388
19968,ZTF23abheflz,350.091240,33.087213


In [7]:
# Vamos a verificar con los oids obtenidos al inicio
size_total = len(oids_total)
oids_unique = np.unique(oids_total)
print('Hay en total', len(oids_unique), 'oids unicos de', size_total, 'oids encontrados en los vetos')

Hay en total 19976 oids unicos de 23227 oids encontrados en los vetos


In [ ]:
# Discrepa el numero de oids unicos originales con los obtenidos, comparemos
oids_dt = df_sin_duplicados['oid'].values
list_oid_non = []

for oid in oids_unique:
  if oid not in oids_dt:
    list_oid_non.append(oid)
    print(oid)

list_oid_non

 
 ZTF24aaqefva
ZTF22abnyduf	
ZTF22absmbfi	
ZTF23aagolfm	
ZTF23aagrzdt	
ZTF23aailyfx 
ZTF23aajduzt	
ZTF23aajvnae	


[np.str_(' '),
 np.str_(' ZTF24aaqefva'),
 np.str_('ZTF22abnyduf\t'),
 np.str_('ZTF22absmbfi\t'),
 np.str_('ZTF23aagolfm\t'),
 np.str_('ZTF23aagrzdt\t'),
 np.str_('ZTF23aailyfx '),
 np.str_('ZTF23aajduzt\t'),
 np.str_('ZTF23aajvnae\t')]

In [ ]:
df_sin_duplicados[df_sin_duplicados['oid'] == 'ZTF22abnyduf']

,oid,ra,dec
85,ZTF22abnyduf,15.016094,21.505762


Entonces podemos concluir que los oids que no aparecen en el DataFrame son en realidad oids mal escritos. Entonces, tenemos todos los objetos necesarios para buscar las imagenes.

In [ ]:
# Guardamos
df_sin_duplicados.to_csv('oids_ra_dec.csv', index=False)

In [ ]:
# Para futuras ejecuciones, leemos el archivo ya
df_sin_duplicados = pd.read_csv('oids_ra_dec.csv')
df_sin_duplicados

,oid,ra,dec
0,ZTF22aaaolwx,230.351159,54.250215
1,ZTF22aaaxmix,235.212955,13.226248
2,ZTF22aabgasm,244.949052,8.071116
3,ZTF22aabwuks,203.220651,29.953070
4,ZTF22aabwuae,202.636665,30.546504
...,...,...,...
19965,ZTF23abhkkcv,20.503617,2.779283
19966,ZTF23abhfhze,13.003263,9.481944
19967,ZTF23abhdcct,259.063642,37.111388
19968,ZTF23abheflz,350.091240,33.087213


Ahora que tenemos correctamente los oids, ra y dec de los objetos y su cantidad coincide con la inicial (23227), se pasa a la recolección de imágenes de PS1.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Usamos DELIGHT para obtener las imagenes PS1
df_img = df_sin_duplicados.copy()
datadir = '/content/drive/MyDrive/data'

dclient = Delight(datadir, df_img.oid.values, df_img.ra.values, df_img.dec.values)

In [ ]:
dclient.download()

KeyboardInterrupt: 

In [ ]:
# Copiar la carpeta '../data' a Drive
!cp -r /content/drive/MyDrive/data /content/drive/MyDrive/

cp: '/content/drive/MyDrive/data' and '/content/drive/MyDrive/data' are the same file


In [ ]:
os.listdir(os.path.join(datadir, "fits"))

['stack_r_ra244.949052_dec8.071116_arcsec120.fits',
 'stack_r_ra230.351159_dec54.250215_arcsec120.fits',
 'stack_r_ra235.212955_dec13.226248_arcsec120.fits',
 'stack_r_ra203.220651_dec29.953070_arcsec120.fits',
 'stack_r_ra202.636665_dec30.546504_arcsec120.fits']